In [ ]:
import sys
sys.path.append('./data')  # Go up one level and into 'data'

In [ ]:
from get_data import load_claim_data # type: ignore

In [8]:
url = "https://data.ny.gov/resource/jshw-gkgu.json"
params = {
    "$limit": "50000"
}

claim_data = load_claim_data(url, params)
claim_data.head()

,accident_ind,accident_date,age_at_injury,alternative_dispute_resolution,ancr_date,assembly_date,atty_rep_ind,average_weekly_wage,birth_year,c2_date,...,ppd_non_scheduled_loss_date,section_32_date,wcio_cause_of_injury_code,wcio_cause_of_injury_desc,wcio_nature_of_injury_code,wcio_nature_of_injury_desc,wcio_pob_code,wcio_pob_desc,controverted_date,ptd_date
0,Y,1999-11-01T00:00:00.000,72,N,2001-12-14T00:00:00.000,2000-01-03T00:00:00.000,Y,200.00,1927,2000-01-06T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Y,1999-08-31T00:00:00.000,43,N,2001-03-30T00:00:00.000,2000-01-03T00:00:00.000,Y,296.51,1955,2000-05-05T00:00:00.000,...,2007-10-22T00:00:00.000,2015-04-14T00:00:00.000,56,LIFTING,52,STRAIN OR TEAR,42,LOWER BACK AREA,NaN,NaN
2,Y,1999-11-01T00:00:00.000,69,N,NaN,2000-01-03T00:00:00.000,Y,0.00,1930,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Y,1999-11-18T00:00:00.000,38,N,2000-08-25T00:00:00.000,2000-01-03T00:00:00.000,Y,250.00,1961,2000-01-28T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Y,1999-12-14T00:00:00.000,33,N,2000-07-10T00:00:00.000,2000-01-03T00:00:00.000,N,980.55,1965,1999-12-22T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
claim_data.head().T

,0,1,2,3,4
accident_ind,Y,Y,Y,Y,Y
accident_date,1999-11-01T00:00:00.000,1999-08-31T00:00:00.000,1999-11-01T00:00:00.000,1999-11-18T00:00:00.000,1999-12-14T00:00:00.000
age_at_injury,72,43,69,38,33
alternative_dispute_resolution,N,N,N,N,N
ancr_date,2001-12-14T00:00:00.000,2001-03-30T00:00:00.000,NaN,2000-08-25T00:00:00.000,2000-07-10T00:00:00.000
assembly_date,2000-01-03T00:00:00.000,2000-01-03T00:00:00.000,2000-01-03T00:00:00.000,2000-01-03T00:00:00.000,2000-01-03T00:00:00.000
atty_rep_ind,Y,Y,Y,Y,N
average_weekly_wage,200.00,296.51,0.00,250.00,980.55
birth_year,1927,1955,1930,1961,1965
c2_date,2000-01-06T00:00:00.000,2000-05-05T00:00:00.000,NaN,2000-01-28T00:00:00.000,1999-12-22T00:00:00.000
